In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import itertools

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [2]:
config_file = '../snakemake/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [39]:
filt_ab = '../'+expand(config['data']['filt_ab'], species='human')[0]
lib_meta = '../'+expand(config['data']['meta'], species='human')[0]
cerberus_h5 = '../'+expand(config['data']['cerb_annot'], species='human')[0]
ca_plus = '../'+expand(config['data']['ca_plus'], species='human')[0]
swan_file = '../'+expand(config['data']['sg'], species='human')[0]
gtf = '../'+expand(config['data']['cerb_gtf'], species='human')[0]

phastcons = '../'+config['ref']['phastcons100']['txt']

gene_subset = 'polya'

biosamp_name_map = '../'+expand(config['ref']['biosamp_map'])[0]

cage_meta = '../'+expand(config['cage']['meta'], species='human')[0]
rampage_meta = '../'+expand(config['rampage']['meta'], species='human')[0]

min_tpm = 1
gene_subset = 'protein_coding'

In [40]:
gtf_df = pr.read_gtf(gtf, rename_attr=True, duplicate_attr=True).as_df()
gtf_df = cerberus.add_stable_gid(gtf_df)
gtf_df = pr.PyRanges(gtf_df)

Found attributes with reserved names: ['source'].
Renaming attributes with suffix '_attr'


In [41]:
gene_df = gtf_df.df
gene_df = gene_df.loc[gene_df.Feature=='gene']
gene_df = gene_df[['gene_id', 'gene_name']]


In [42]:
## mean intron length
introns = gtf_df.features.introns(by='transcript').df
introns['len'] = (introns.Start-introns.End).abs()
temp = introns[['gene_id', 'transcript_id', 'len']].groupby(['gene_id', 'transcript_id']).sum().reset_index()
temp = temp[['gene_id', 'len']].groupby('gene_id').mean().reset_index().rename({'len':'mean_intron_len'}, axis=1)
gene_df = gene_df.merge(temp, how='left', on='gene_id')

In [43]:
gene_df.head()

,gene_id,gene_name,mean_intron_len
0,ENSG00000000460,C1orf112,51925.666667
1,ENSG00000000971,CFH,65430.333333
2,ENSG00000001461,NIPAL3,40274.700000
3,ENSG00000004487,KDM1A,60957.750000
4,ENSG00000007933,FMO3,16727.250000


In [44]:
## mean # introns
temp = introns[['gene_id', 'transcript_id', 'len']].groupby(['gene_id', 'transcript_id']).count().reset_index()
temp = temp[['gene_id', 'len']].groupby('gene_id').mean().reset_index().rename({'len':'mean_n_intron'}, axis=1)
gene_df = gene_df.merge(temp, how='left', on='gene_id')

In [45]:
gene_df.head()

,gene_id,gene_name,mean_intron_len,mean_n_intron
0,ENSG00000000460,C1orf112,51925.666667,21.666667
1,ENSG00000000971,CFH,65430.333333,13.333333
2,ENSG00000001461,NIPAL3,40274.700000,8.800000
3,ENSG00000004487,KDM1A,60957.750000,17.875000
4,ENSG00000007933,FMO3,16727.250000,6.250000


In [46]:
## exon len % 3 (internal exons only)
gtf_df = gtf_df.as_df()
exons = gtf_df.loc[gtf_df.Feature=='exon']

# remove monoexonic or two-exon transcripts
temp = exons[['transcript_id', 'Start']].groupby('transcript_id').count().reset_index().rename({'Start': 'n_exons'}, axis=1)
temp = temp.loc[temp.n_exons>2]
exons = exons.loc[exons.transcript_id.isin(temp.transcript_id.tolist())]

# restrict to internal exons
exons = exons.reset_index(drop=True)
drop_inds = exons[['transcript_id']].drop_duplicates(keep='first').index.tolist()+\
            exons[['transcript_id']].drop_duplicates(keep='last').index.tolist()
keep_inds = list(set(exons.index.tolist())-set(drop_inds))
exons = exons.loc[keep_inds]

# compute len of exons
exons['exon_len'] = (exons.Start-exons.End).abs()

# mod 3 info
exons['exon_len_mod_3'] = exons.exon_len % 3 
exons['frame_preserving'] = exons.exon_len % 3 == 0

# % of frame preserving exons / gene
keep_cols = ['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'exon_len_mod_3', 'frame_preserving']
exons = exons[keep_cols].drop_duplicates()
exons = exons[['gene_id', 'frame_preserving']].groupby('gene_id').agg({'frame_preserving': ['count', 'sum']}).reset_index().rename({'count':'n_exons', 'sum':'n_frame_preserving_exons'}, axis=1)
exons.columns = ['gene_id', 'n_exons', 'n_frame_preserving_exons']
exons['perc_frame_pres_exons'] = (exons.n_frame_preserving_exons/exons.n_exons)*100
gene_df = gene_df.merge(exons, how='left', on='gene_id')

In [47]:
gene_df.head()

,gene_id,gene_name,mean_intron_len,mean_n_intron,n_exons,n_frame_preserving_exons,perc_frame_pres_exons
0,ENSG00000000460,C1orf112,51925.666667,21.666667,25.0,13.0,52.000000
1,ENSG00000000971,CFH,65430.333333,13.333333,23.0,19.0,82.608696
2,ENSG00000001461,NIPAL3,40274.700000,8.800000,11.0,2.0,18.181818
3,ENSG00000004487,KDM1A,60957.750000,17.875000,23.0,5.0,21.739130
4,ENSG00000007933,FMO3,16727.250000,6.250000,7.0,2.0,28.571429


In [48]:
## conservation

min_cons_score = 250

# read conserved elements in, threshold for a certain score
cons = pd.read_csv(phastcons, sep='\t', header=None,
                 names=['bin', 'Chromosome', 'Start', 'End', 'idk1', 'score'])
cons = cons[['Chromosome', 'Start', 'End', 'score']]
cons = cons.loc[cons.score >= min_cons_score]
cons = pr.PyRanges(cons)

# get exons
exons = gtf_df[['Chromosome', 'Start', 'End', 'gene_name',
                 'gene_id', 'transcript_id', 'Feature']]
exons = exons.loc[gtf_df.Feature=='exon']
exons['exon_len'] = (exons.Start-exons.End).abs()
exons = pr.PyRanges(exons)
exons = exons.join(cons, report_overlap=True)

# get the % length covered in each thing
exons = exons.df
keep_cols = ['Chromosome', 'Start', 'End', 'gene_name', 'gene_id', 'transcript_id', 'exon_len', 'Overlap']
gb_cols = ['Chromosome', 'Start', 'End', 'gene_name', 'gene_id', 'transcript_id', 'exon_len']
temp = exons[keep_cols].groupby(gb_cols, observed=True).sum().reset_index().rename({'Overlap': 'cons_len'}, axis=1)

# get total len of transcript and len covered of each transcript 
keep_cols = ['gene_name', 'gene_id', 'transcript_id', 'exon_len', 'cons_len']
gb_cols = ['gene_name', 'gene_id', 'transcript_id']
temp = temp[keep_cols].groupby(gb_cols,observed=True).sum().reset_index().rename({'exon_len': 't_len'}, axis=1)

# now get mean % exonic conservation per gene
temp['perc_cons'] = (temp['cons_len']/temp['t_len'])*100
keep_cols = ['gene_id', 'perc_cons']
gb_cols = ['gene_id']
temp = temp[keep_cols].groupby(gb_cols, observed=True).mean().reset_index().rename({'perc_cons':'mean_perc_cons'}, axis=1)
gene_df = gene_df.merge(temp, how='left', on='gene_id')

In [49]:
gene_df.head()

,gene_id,gene_name,mean_intron_len,mean_n_intron,n_exons,n_frame_preserving_exons,perc_frame_pres_exons,mean_perc_cons
0,ENSG00000000460,C1orf112,51925.666667,21.666667,25.0,13.0,52.000000,61.028481
1,ENSG00000000971,CFH,65430.333333,13.333333,23.0,19.0,82.608696,26.370211
2,ENSG00000001461,NIPAL3,40274.700000,8.800000,11.0,2.0,18.181818,38.732814
3,ENSG00000004487,KDM1A,60957.750000,17.875000,23.0,5.0,21.739130,70.077454
4,ENSG00000007933,FMO3,16727.250000,6.250000,7.0,2.0,28.571429,34.694776


In [50]:
temp.head()

,gene_id,mean_perc_cons
0,ENSG00000000003,41.472387
1,ENSG00000000005,79.513616
2,ENSG00000000419,57.704859
3,ENSG00000000457,55.789980
4,ENSG00000000460,61.028481
